# Inspectable execution tests

In [1]:
import numpy as np
np.random.seed(1)
from pv_evaluation.metrics import pairwise_precision_recall, cluster_precision_recall, cluster_recall, cluster_precision
from pv_evaluation.estimators import pairwise_precision_estimator, pairwise_recall_estimator, cluster_precision_estimator, cluster_recall_estimator
from pv_evaluation.benchmark import load_als_inventors_benchmark

In [2]:
reference = load_als_inventors_benchmark()

# Create prediction membership vector as a noisy version of the reference
prediction = reference.copy()
for _ in range(10000):
    i,j = np.random.randint(0, len(prediction), size=2)
    prediction[i] = prediction[j]

True pairwise precision and recall:

In [3]:
P, R = pairwise_precision_recall(prediction, reference)

print(P, R)

0.8495857333565273 0.864769100993944


True cluster precision and recall:

In [4]:
P_cluster, R_cluster = cluster_precision_recall(prediction, reference)

print(P_cluster, R_cluster)

0.8649243999577063 0.9004875736439358


## Sampling clusters with probability proportional to their size

In [5]:
def sample_clusters(reference, sample_size):
    clusters = np.random.choice(reference.values, size=sample_size, replace=True)
    return reference[reference.isin(clusters)]

Checking for unbiasedness.

In [6]:
k = 50

for params in [("record", "uniform"), ("cluster", "cluster_size"), ("cluster_block", "cluster_size")]:
    P_hat = np.zeros(k)
    R_hat = np.zeros(k)
    for i in range(k):
        sample = sample_clusters(reference, sample_size=200)
        P_hat[i] = pairwise_precision_estimator(prediction, sample, sampling_type=params[0], weights=params[1])
        R_hat[i] = pairwise_recall_estimator(prediction, sample, sampling_type=params[0], weights=params[1])

    print(np.mean(P_hat), np.mean(R_hat))
    assert np.abs(np.mean(P_hat) - P) < 0.05
    assert np.abs(np.mean(R_hat) - R) < 0.05

0.8286886262259623 0.8659868339137509
0.8130723402604646 0.8665343519807581
0.8432463003236158 0.8630406187342097


In [7]:
k = 100

for params in [("cluster_block", "cluster_size")]:
    P_hat = np.zeros(k)
    R_hat = np.zeros(k)
    for i in range(k):
        sample = sample_clusters(reference, sample_size=200)
        P_hat[i] = cluster_precision_estimator(prediction, sample, sampling_type=params[0], weights=params[1])
        R_hat[i] = cluster_recall_estimator(prediction, sample, sampling_type=params[0], weights=params[1])

    print(np.mean(P_hat), np.mean(R_hat))
    assert np.abs(np.mean(P_hat) - P_cluster) < 0.05
    assert np.abs(np.mean(R_hat) - R_cluster) < 0.05

0.8417909754739342 0.8918661171483601


## Sampling clusters uniformly at random

In [8]:
def sample_clusters_uniformly(reference, sample_size):
    clusters = np.random.choice(reference.unique(), size=sample_size, replace=True)
    return reference[reference.isin(clusters)]

Checking for (near) unbiasedness. Note that bias of these ratio estimators would be high in small samples.

In [9]:
k = 50

for params in [("cluster", "uniform"), ("cluster_block", "uniform")]:
    P_hat = np.zeros(k)
    R_hat = np.zeros(k)

    for i in range(k):
        sample = sample_clusters_uniformly(reference, sample_size=400)
        P_hat[i] = pairwise_precision_estimator(prediction, sample, sampling_type=params[0], weights=params[1])
        R_hat[i] = pairwise_recall_estimator(prediction, sample, sampling_type=params[0], weights=params[1])

    print(np.mean(P_hat), np.mean(R_hat))
    assert np.abs(np.mean(P_hat) - P) < 0.05
    assert np.abs(np.mean(R_hat) - R) < 0.05

0.8228562617126128 0.8604932548282794
0.836117074275537 0.8676234951792299


In [10]:
k = 50

for params in [("cluster_block", "uniform")]:
    P_hat = np.zeros(k)
    R_hat = np.zeros(k)
    for i in range(k):
        sample = sample_clusters_uniformly(reference, sample_size=200)
        P_hat[i] = cluster_precision_estimator(prediction, sample, sampling_type=params[0], weights=params[1])
        R_hat[i] = cluster_recall_estimator(prediction, sample, sampling_type=params[0], weights=params[1]) 
    
    print(np.mean(P_hat), np.mean(R_hat))
    assert np.abs(np.mean(P_hat) - P_cluster) < 0.05
    assert np.abs(np.mean(R_hat) - R_cluster) < 0.05

0.8528641966295608 0.905009585807827
